# Fine-tune Amazon Titan model in Bedrock for summarization task

## Import libraries

In [1]:
!pip install --upgrade sagemaker datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.1 MB/s eta 0:00:00
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 30.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prepare the data in Bedrock required format

In [2]:
import json
from datasets import load_dataset

# Here we are using the Dolly dataset for fine-tuning the titan model for summarization task
dolly_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Filter the dataset to include only summarization examples
summarization_dataset = dolly_dataset.filter(lambda example: example["category"] == "summarization")
summarization_dataset = summarization_dataset.remove_columns("category")

# Create a new DataFrame with Bedrock supported format
modified_df = summarization_dataset.map(
    lambda example: {"prompt": f"{example['instruction']} {example['context']}",
                     "completion": example["response"]}
)

# Set max length to support Bedrock Customization Max token Length Quota
max_length_per_row = 18000

# Set max length to support Bedrock Customization Max token Length Quota
max_row_count = 9000

# Define a function to check if the total length of prompt and completion is within the specified max_length
def within_length(example):
    return len(example['prompt'] + example['completion']) <= max_length_per_row

# Filter the DataFrame to include only examples within the specified max length
modified_df = modified_df.filter(within_length)

num_rows = len(modified_df)
print(f"Number of rows: {num_rows}")


# Take the maximum supported dataset size
if num_rows > max_row_count:
    modified_df = modified_df.select(range(max_row_count)) 

# Remove unnecessary columns
modified_df = modified_df.remove_columns(["instruction", "context", "response"])

# Dump the modified DataFrame to a JSON file "train.jsonl" in local directory
modified_df.to_json("train.jsonl", orient="records", lines=True)

# Print the 11th example in the modified DataFrame (since Python uses 0-based indexing)
print(modified_df[10])



/Users/sascha/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filter: 100%|██████████| 1188/1188 [00:00<00:00, 121408.15 examples/s]


Number of rows: 1183


Creating json from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 77.24ba/s]

{'prompt': 'Please describe  what is oil and give me a list of it’s applications. An oil is any nonpolar chemical substance that is composed primarily of hydrocarbons and is hydrophobic (does not mix with water) & lipophilic (mixes with other oils). Oils are usually flammable and surface active. Most oils are unsaturated lipids that are liquid at room temperature.\n\nThe general definition of oil includes classes of chemical compounds that may be otherwise unrelated in structure, properties, and uses. Oils may be animal, vegetable, or petrochemical in origin, and may be volatile or non-volatile. They are used for food (e.g., olive oil), fuel (e.g., heating oil), medical purposes (e.g., mineral oil), lubrication (e.g. motor oil), and the manufacture of many types of paints, plastics, and other materials. Specially prepared oils are used in some religious ceremonies and rituals as purifying agents.', 'completion': 'An oil is a chemical substance that is composed primarily of hydrocarbons

## Upload training data to S3

In [3]:
import boto3
from sagemaker import Session

# Update this to your bucket name and make sure it exists in S3
default_bucket = 'bedrock-doit-demo'

# Create a session using the provided AWS SDK sessions and default bucket
session = Session(boto_session=boto3.session.Session(),
                  sagemaker_client=boto3.client('sagemaker'),
                  sagemaker_runtime_client=boto3.client('runtime.sagemaker'),
                  default_bucket=default_bucket)

# Create an S3 resource using the AWS SDK
s3 = boto3.resource('s3')

# Specify the path to the training data on your local machine
train_data_path = 'train.jsonl'

# Upload the training data to the specified S3 key prefix 'PreProcessed'
s3_train_data = session.upload_data(path=train_data_path, key_prefix='fine-tuning')

# Print a message indicating the successful upload
print(f"Uploaded {train_data_path} to {s3_train_data}")


Uploaded train.jsonl to s3://bedrock-doit-demo/fine-tuning/train.jsonl


## Create the bedrock training job

In [4]:
import boto3
import uuid  # Import the 'uuid' module for generating a unique identifier
from sagemaker import get_execution_role

# Initialize Bedrock client
bedrock = boto3.client(service_name='bedrock')

# Get the SageMaker execution role
#role = get_execution_role()

# Set parameters
customizationType = "FINE_TUNING"

# Base model to use
basemodelId = 'amazon.titan-text-express-v1'

# Model ID for provisioned throughput
# https://docs.aws.amazon.com/bedrock/latest/userguide/prov-thru-api.html
baseModelIdentifierForProvisonedThroughput = "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1:0:8k"
job_prefix = "customTitan"

# Update this to a valid roleARn
roleArn = 'arn:aws:iam::303673948954:role/bedrock_dev'

# Generate a unique identifier for the job and custom model name
job_uuid = str(uuid.uuid4())[:8]  # Extracting the first 8 characters for brevity
jobName = f"{job_prefix}-{job_uuid}"
customModelName = f"{job_prefix}-{job_uuid}"

hyperParameters = {
    "epochCount": "2",
    "batchSize": "1",
    "learningRate": "0.00001",
}

# Specify the training data configuration using the previously uploaded S3 data
trainingDataConfig = {"s3Uri": s3_train_data}

# Specify the output data configuration for the custom model
outputDataConfig = {"s3Uri": f"s3://{default_bucket}/CustomModel/"}

# Create a job for model customization
jobIdentifier = bedrock.create_model_customization_job(
    jobName=jobName,
    customModelName=customModelName,
    roleArn=roleArn,
    baseModelIdentifier=baseModelIdentifierForProvisonedThroughput,
    hyperParameters=hyperParameters,
    trainingDataConfig=trainingDataConfig,
    outputDataConfig=outputDataConfig
)

# Print the identifier for the created job
print(f"Model customization job created with identifier: {jobIdentifier}")


ParamValidationError: Parameter validation failed:
Missing required parameter in input: "roleArn"

## Monitor the job till the status is shown as "Completed"

In [1]:
fine_tune_job = bedrock.get_model_customization_job(jobIdentifier=jobIdentifier['jobArn'])
print(fine_tune_job['status'])

NameError: name 'bedrock' is not defined

## Create provisioned no-commit throughput for the custom model (Only run the following once the status of the above job is shown as "Completed")

In [ ]:
customModelId=fine_tune_job['outputModelArn']


provisionedModelName = f"{job_prefix}-provisioned-{job_uuid}"

# Create the provisioned capacity without passing any commitment option
provisionedModelArn = bedrock.create_provisioned_model_throughput(
    modelUnits=1,
    provisionedModelName=provisionedModelName, 
    modelId=customModelId
   )['provisionedModelArn']

## Check the provisoned capacity creation status

In [ ]:
# Get Provisioned model status untill it's completed
provisionedModelStatus = bedrock.get_provisioned_model_throughput(provisionedModelId=provisionedModelArn)
print (provisionedModelStatus['status'])

## Run inference on the custom provisioned model and the base model via bedrock and observe the difference

In [ ]:
import json

# Initialize Bedrock Runtime client in the specified region
bedrockRuntime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Sample request body containing text for summarization and parameters for model inference
body = json.dumps({
    "inputText": "Summarize the following:   TOKYO–January 19, 2024–Today, Amazon Web Services (AWS) announced its plans to invest 2.26 trillion yen into its existing cloud infrastructure in Tokyo and Osaka by 2027 to meet growing customer demand for cloud services in Japan. According to the new AWS Economic Impact Study (EIS) for Japan, this planned investment is estimated to contribute 5.57 trillion yen to Japan’s Gross Domestic Product (GDP), and support an estimated average of 30,500 full-time equivalent (FTE) jobs in local Japanese businesses each year. Having already invested 1.51 trillion yen in Japan from 2011 to 2022, AWS’s planned total investment into cloud infrastructure in the country by 2027 will be approximately 3.77 trillion yen. Hundreds of thousands of active customers use the two AWS Regions in Japan to digitally transform (DX) their businesses. AWS opened its first office in Japan in 2009 and launched the AWS Asia Pacific (Tokyo) Region in 2011, and the AWS Asia Pacific (Osaka) Region in 2021. As demand for cloud services to drive the government’s DX agenda grew in Japan, AWS invested 1.51 trillion yen between 2011 and 2022 to construct, connect, operate, and maintain AWS data centers. This is estimated to have contributed 1.46 trillion yen to Japan’s GDP and supported more than 7,100 FTE jobs. These positions, including construction, facility maintenance, engineering, telecommunications, and other jobs within the country’s broader economy, are part of the AWS data center supply chain in Japan.",
    "textGenerationConfig": {
        "temperature": 0.01,  
        "topP": 0.99,
        "maxTokenCount": 300
    }
})

# Specify content types for request and response
accept = 'application/json'
contentType = 'application/json'

# Invoke custom model with the provided parameters
response = bedrockRuntime.invoke_model(body=body, modelId=provisionedModelArn, accept=accept, contentType=contentType)

# Parse and print the output from the custom model
response_body_custom = json.loads(response.get('body').read())
print("Custom Model Output:")
print(response_body_custom['results'][0]['outputText'])

# Invoke the base model with the same parameters
response = bedrockRuntime.invoke_model(body=body, modelId=basemodelId, accept=accept, contentType=contentType)

# Parse and print the output from the base model
response_body_base = json.loads(response.get('body').read())
print("\n")
print("Base Model Output:")
print(response_body_base['results'][0]['outputText'])


## Delete the provisioned capacity and the custom model

In [ ]:
# Delete the provisioned capacity
bedrock.delete_provisioned_model_throughput(provisionedModelId=provisionedModelArn)

# Delete the custom model
bedrock.delete_custom_model (modelIdentifier=customModelId)
